In [1]:
from config import API_Key
from config import gkey

import pandas as pd
import geopandas as gpd

import requests
import json
from citipy import citipy

In [2]:
url = f"http://api.airvisual.com/v2/city?city=Los Angeles&state=California&country=USA&key={API_Key}"
response=requests.get(url)
data=response.json()
print(json.dumps(data,indent=4,sort_keys=4))

{
    "data": {
        "city": "Los Angeles",
        "country": "USA",
        "current": {
            "pollution": {
                "aqicn": 37,
                "aqius": 80,
                "maincn": "p2",
                "mainus": "p2",
                "ts": "2022-06-16T12:00:00.000Z"
            },
            "weather": {
                "hu": 85,
                "ic": "04n",
                "pr": 1009,
                "tp": 18,
                "ts": "2022-06-16T12:00:00.000Z",
                "wd": 0,
                "ws": 0
            }
        },
        "location": {
            "coordinates": [
                -118.2417,
                34.0669
            ],
            "type": "Point"
        },
        "state": "California"
    },
    "status": "success"
}


In [3]:
filepath="Climate_Data/ClimateClassifications.csv"
climate_df=pd.read_csv(filepath)
climate_df

,Lat,Lon,Cls
0,-89.75,-179.75,EF
1,-89.75,-179.25,EF
2,-89.75,-178.75,EF
3,-89.75,-178.25,EF
4,-89.75,-177.75,EF
...,...,...,...
92411,83.75,-30.75,ET
92412,83.75,-30.25,ET
92413,83.75,-29.75,ET
92414,83.75,-29.25,ET


In [4]:
geo_climate_df=gpd.GeoDataFrame(climate_df,geometry=gpd.points_from_xy(climate_df.Lon, climate_df.Lat))
geo_climate_df.head()

,Lat,Lon,Cls,geometry
0,-89.75,-179.75,EF,POINT (-179.75000 -89.75000)
1,-89.75,-179.25,EF,POINT (-179.25000 -89.75000)
2,-89.75,-178.75,EF,POINT (-178.75000 -89.75000)
3,-89.75,-178.25,EF,POINT (-178.25000 -89.75000)
4,-89.75,-177.75,EF,POINT (-177.75000 -89.75000)


In [5]:
target_city = "Melbourne, Australia"

target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={target_city}&key={gkey}"
geo_data = requests.get(target_url).json()

# Extract latitude and longitude
lat = geo_data["results"][0]["geometry"]["location"]["lat"]
lng = geo_data["results"][0]["geometry"]["location"]["lng"]

# Print the latitude and longitude
print('''
    City: {0}
    Latitude: {1}
    Longitude: {2}
    '''.format(target_city, lat, lng))


    City: Melbourne, Australia
    Latitude: -37.8136276
    Longitude: 144.9630576
    


In [6]:
lat_long={}
lat_long["lat"]=lat
lat_long["lng"]=lng
lat_long["target_city"]=target_city


place_df=pd.DataFrame([lat_long])
place_df


,lat,lng,target_city
0,-37.813628,144.963058,"Melbourne, Australia"


In [7]:
geo_place_df=gpd.GeoDataFrame(place_df,geometry=gpd.points_from_xy(place_df.lng, place_df.lat))
geo_place_df.head()

,lat,lng,target_city,geometry
0,-37.813628,144.963058,"Melbourne, Australia",POINT (144.96306 -37.81363)


In [8]:
geo_place_df.sjoin_nearest(geo_climate_df, how='inner')

NotImplementedError: Currently, only PyGEOS >= 0.10.0 supports `nearest_all`. To use PyGEOS within GeoPandas, you need to install PyGEOS: 'conda install pygeos' or 'pip install pygeos'

In [ ]:
place={}
place["Melb"]={}
place["Melb"]["Lat"]=-37.8
place["Melb"]["Lon"]=144.9
place

place_df=pd.DataFrame.from_dict(place, orient='index')
place_df


,Lat,Lon
Melb,-37.8,144.9
